# 15-9. 프로젝트 - Movielens 영화 추천 실습


15-9. 프로젝트 - Movielens 영화 추천 실습

이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

Cloud shell에서 아래 명령어를 입력해 주세요.

$ mkdir -p ~/aiffel/recommendata_iu/data/ml-1m  

$ ln -s ~/data/ml-1m/* ~/aiffel/recommendata_iu/data/ml-1m

# 1) 데이터 준비와 전처리
***
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
# 라이브러리from implicit.als import AlternatingLeastSquares 준비

import pandas as pd
import os
import numpy as np

import scipy
from scipy.sparse import csr_matrix
import implicit
from implicit.als import AlternatingLeastSquares

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']    #컬럼명 지정
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(20)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [7]:
# 영화를 title로 입력할 것이기때문에 title이란 컬럼을 불러와야합니다.
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings


,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


# 2) 분석해 봅시다.
***
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [8]:
# ratings에 있는 유니크한 영화 개수

ratings['movie_id'].nunique()

3628

In [9]:
# ratings에 있는 유니크한 사용자 수

ratings['user_id'].nunique()

6039

In [10]:
# 가장 인기 있는 영화 30개(인기순)

movie_count = ratings.groupby('movie_id')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: counts, dtype: int64

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
***


In [11]:
# 내가 선호하는 영화를 5가지 골라서 rating에 추가
my_favorite = ['minority report', 'mission impossible', 'get out' , 'inception', 'gravity']

my_list = pd.DataFrame({'user_id': ['joonho']*5, 'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['joonho']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)       

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,joonho,NaN,5,NaN,minority report
1,joonho,NaN,5,NaN,mission impossible
2,joonho,NaN,5,NaN,get out
3,joonho,NaN,5,NaN,inception
4,joonho,NaN,5,NaN,gravity


In [12]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [13]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['joonho'])    
print(movie_to_idx['inception'])

6039
3631


In [14]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,3628,5,NaN,minority report
1,6039,3629,5,NaN,mission impossible
2,6039,3630,5,NaN,get out
3,6039,3631,5,NaN,inception


# 4) CSR matrix를 직접 만들어 봅시다.


In [15]:
# csr_matrix


num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.


In [16]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

# Implicit AlternatingLeastSquares 모델의 선언
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 
# 3. use_gpu : GPU를 사용할 것인지 
# 4. iterations : epochs와 같은 의미입니다.
als_model = AlternatingLeastSquares(factors=2000, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

In [19]:
joonho, inception = user_to_idx['joonho'], movie_to_idx['inception']
joonho_vector, inception_vector = als_model.user_factors[joonho], als_model.item_factors[inception]

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


In [20]:
# 내적하는 코드
np.dot(joonho_vector, inception_vector)

0.6091608

In [21]:
getout = movie_to_idx['get out']
getout_vector = als_model.item_factors[getout]
np.dot(joonho_vector, getout_vector)

0.6043261

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [22]:
# (id, 유사도 )
favorite_movie = 'inception'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 1.0),
 (3386, 0.9399484),
 (3288, 0.93677014),
 (3404, 0.9355735),
 (3630, 0.9341082),
 (3632, 0.93327785),
 (3550, 0.9309863),
 (2985, 0.9304288),
 (3628, 0.93005913),
 (3545, 0.92990786),
 (3148, 0.92977536),
 (3241, 0.9291588),
 (3574, 0.9288664),
 (3629, 0.9277823),
 (3529, 0.9274098)]

In [23]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['inception',
 'Beauty (1998)',
 'Hatchet For the Honeymoon (Rosso Segno Della Follia) (1969)',
 'Born to Win (1971)',
 'get out',
 'gravity',
 'Jerry & Tom (1998)',
 "Billy's Holiday (1995)",
 'minority report',
 'Ten Benny (1997)',
 'Defying Gravity (1997)',
 'American Strays (1996)',
 'Century (1993)',
 'mission impossible',
 'Buck and the Preacher (1972)']

In [24]:
# 비슷한 영화 이름 검색 함수
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [25]:
get_similar_movie('get out')

['get out',
 'Beauty (1998)',
 'inception',
 'Hatchet For the Honeymoon (Rosso Segno Della Follia) (1969)',
 'Ten Benny (1997)',
 'gravity',
 'Defying Gravity (1997)',
 'American Strays (1996)',
 'Born to Win (1971)',
 "Billy's Holiday (1995)"]

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [26]:
user = user_to_idx['joonho']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(2101, 0.13456476),
 (2139, 0.13414857),
 (2543, 0.13384911),
 (2335, 0.12679817),
 (2930, 0.118230715),
 (2360, 0.11057481),
 (2287, 0.106952816),
 (2487, 0.10640154),
 (2847, 0.10602918),
 (1690, 0.102860354),
 (2079, 0.10278614),
 (1623, 0.102375925),
 (3067, 0.09951097),
 (3422, 0.09826024),
 (1057, 0.097966924),
 (3593, 0.09692123),
 (2890, 0.09472403),
 (2731, 0.09459311),
 (206, 0.09420456),
 (3302, 0.094092384)]

In [27]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Palm Beach Story, The (1942)',
 'Urban Legends: Final Cut (2000)',
 'Brandon Teena Story, The (1998)',
 'Affair of Love, An (Une Liaison Pornographique) (1999)',
 'Vagabond (Sans toit ni loi) (1985)',
 'Beverly Hillbillies, The (1993)',
 'Poison Ivy: New Seduction (1997)',
 'Life (1999)',
 'Go West (1925)',
 'Ridicule (1996)',
 'Great Mouse Detective, The (1986)',
 'Pale Rider (1985)',
 'Lodger, The (1926)',
 'Country Life (1994)',
 'Escape from L.A. (1996)',
 'Sonic Outlaws (1995)',
 'Under Suspicion (2000)',
 'Slingshot, The (Kådisbellan ) (1993)',
 'Love! Valour! Compassion! (1997)',
 'Glass Shield, The (1994)']

# 회고
***


Implicit 라이브러리를 활용하여 Matrix Factorization(이하 MF) 기반의 추천 모델을 만들어 CSR Matrix를 이용해 추천 모델을 만들었는데 영화 제목 추천을 위해 새로 한 모델링에서 적합 추천도가 너무 낮게 나와 여러 방법으로 학습방법을 바꿔보았으나 알 수 없었다.  
Explicit Datasets란 유저의 호불호를 알 수 있는 명시적 데이터로써 유용하나 데이터수집이 어렵다  
Implicit Datasets은 암묵적으로 선호할 것이다라고 추측할 수 있는 데이터로 검색 기록, 방문 페이지, 구매 내역 등이 있음
부정적인 피드백이 없다(No Negative Feedback)  
애초에 잡음이 많다(Inherently Noisy)  
수치는 신뢰도를 의미한다.  
생각보다 옛날 영화들이 많이 추천된 것을 확인하였고 factors와 iterations의 수치를 조절하는 것 외에는 모델의 성능에 영향을 미치는 요인이 없는 것 같다.